In [37]:
import requests
import pandas as pd
from datetime import datetime
import warnings
warnings.filterwarnings('ignore')
from pandas import json_normalize


<h2> Info Cơ bản </h2>

In [6]:
import pandas as pd

def listing_companies(live=False, source='Wifeed'):
    if live:
        if source == 'Wifeed':
            df = live_stock_list()
        elif source == 'SSI':
            df = organ_listing()
    else:
        path = 'https://raw.githubusercontent.com/thinh-vu/vnstock/beta/data/listing_companies_enhanced-2023.csv'
        df = pd.read_csv(path)

    return df


In [11]:
df=listing_companies()

In [12]:
df.columns

Index(['ticker', 'comGroupCode', 'organName', 'organShortName',
       'organTypeCode', 'comTypeCode', 'icbName', 'icbNamePath', 'sector',
       'industry', 'group', 'subgroup', 'icbCode', 'VN30', 'VNMID', 'VN100',
       'VNSML', 'VNALL', 'HNX30', 'VNX50', 'VNXALL', 'VNDIAMOND', 'VNFINLEAD',
       'VNFINSELECT', 'VNSI', 'VNCOND', 'VNCONS', 'VNENE', 'VNFIN', 'VNHEAL',
       'VNIND', 'VNIT', 'VNMAT', 'VNREAL', 'VNUTI'],
      dtype='object')

In [28]:
dfInfo = df[['ticker', 'comGroupCode', 'organName', 'organShortName', 'comTypeCode', 'icbName', 'icbNamePath', 'sector', 'industry']]
dfInfo['updateAt'] = datetime.now()
dfInfo

,ticker,comGroupCode,organName,organShortName,comTypeCode,icbName,icbNamePath,sector,industry,updateAt
0,SSI,HOSE,Công ty Cổ phần Chứng khoán SSI,Chứng khoán SSI,CK,Môi giới chứng khoán,Tài chính/Dịch vụ tài chính/Dịch vụ tài chính/...,Tài chính,Dịch vụ tài chính,2023-11-01 00:22:25.593979
1,BCM,HOSE,Tổng Công ty Đầu tư và Phát triển Công nghiệp ...,Becamex IDC,CT,Bất động sản,Tài chính/Bất động sản/Bất động sản/Bất động sản,Tài chính,Bất động sản,2023-11-01 00:22:25.593979
2,VHM,HOSE,Công ty Cổ phần Vinhomes,Vinhomes,CT,Bất động sản,Tài chính/Bất động sản/Bất động sản/Bất động sản,Tài chính,Bất động sản,2023-11-01 00:22:25.593979
3,VIC,HOSE,Tập đoàn Vingroup - Công ty CP,VinGroup,CT,Bất động sản,Tài chính/Bất động sản/Bất động sản/Bất động sản,Tài chính,Bất động sản,2023-11-01 00:22:25.593979
4,VRE,HOSE,Công ty Cổ phần Vincom Retail,Vincom Retail,CT,Bất động sản,Tài chính/Bất động sản/Bất động sản/Bất động sản,Tài chính,Bất động sản,2023-11-01 00:22:25.593979
...,...,...,...,...,...,...,...,...,...,...
1602,SDK,UPCOM,Công ty Cổ phần Cơ khí Luyện Kim,Cơ khí Luyện Kim,CT,Máy công nghiệp,Công nghiệp/Hàng & Dịch vụ Công nghiệp/Công ng...,Công nghiệp,Hàng & Dịch vụ Công nghiệp,2023-11-01 00:22:25.593979
1603,SRF,HOSE,Công ty Cổ phần Searefico,SEAREFICO,CT,Máy công nghiệp,Công nghiệp/Hàng & Dịch vụ Công nghiệp/Công ng...,Công nghiệp,Hàng & Dịch vụ Công nghiệp,2023-11-01 00:22:25.593979
1604,UEM,UPCOM,Công ty Cổ phần Cơ điện Uông Bí - Vinacomin,Cơ điện Uông Bí - Vinacomin,CT,Máy công nghiệp,Công nghiệp/Hàng & Dịch vụ Công nghiệp/Công ng...,Công nghiệp,Hàng & Dịch vụ Công nghiệp,2023-11-01 00:22:25.593979
1605,VEA,UPCOM,Tổng Công ty Máy động lực và Máy nông nghiệp V...,Máy động lực và Máy NN,CT,Máy công nghiệp,Công nghiệp/Hàng & Dịch vụ Công nghiệp/Công ng...,Công nghiệp,Hàng & Dịch vụ Công nghiệp,2023-11-01 00:22:25.593979


In [26]:
import pandas as pd
import pyodbc
from sqlalchemy import create_engine


df=dfInfo
df = df.replace('-', 0)


conn = pyodbc.connect('Driver={SQL Server};'
                      'Server=MSI\\SQLEXPRESS;'
                      'Database=dataFinance;'
                      'Trusted_Connection=yes;')

cursor = conn.cursor()

for index, row in df.iterrows():
    cursor.execute("INSERT INTO Info (ticker, nameExchange, organName, organShortName, comTypeCode, icbName, icbNamePath, sector, industry, updateAt) VALUES (?, ?, ?, ?, ?, ?, ?, ?, ?, ?)",
                   row['ticker'], row['comGroupCode'], row['organName'], row['organShortName'], row['comTypeCode'], row['icbName'], row['icbNamePath'], row['sector'], row['industry'], row['updateAt'])

conn.commit()

conn.close()



<h2> Detail Info </h2>

In [29]:
def company_overview (symbol):
    data = requests.get(f'https://apipubaws.tcbs.com.vn/tcanalysis/v1/ticker/{symbol}/overview').json()
    df = json_normalize(data)
    df = df[['ticker', 'exchange', 'industry', 'companyType',
            'noShareholders', 'foreignPercent', 'outstandingShare', 'issueShare',
            'establishedYear', 'noEmployees',  
            'stockRating', 'deltaInWeek', 'deltaInMonth', 'deltaInYear', 
            'shortName', 'industryEn', 'industryID', 'industryIDv2', 'website']]
    return df

In [39]:
import pyodbc

conn = pyodbc.connect('Driver={SQL Server};'
                      'Server=MSI\\SQLEXPRESS;'
                      'Database=dataFinance;'
                      'Trusted_Connection=yes;')

cursor = conn.cursor()

sql_query = "SELECT DISTINCT ticker FROM Info;"

cursor.execute(sql_query)

tickers = [row.ticker for row in cursor]

conn.close()



In [42]:
data_list = []

for ticker in tickers:
    try:
        result = company_overview(ticker)
        data_list.append(result)
    except Exception as e:
        continue
df = pd.concat(data_list, ignore_index=True)

# In ra DataFrame
print(df)

     ticker exchange                 industry companyType  noShareholders  \
0       A32    UPCOM  Hàng cá nhân & Gia dụng          CT             488   
1       AAA     HOSE                 Hóa chất          CT           27697   
2       AAM     HOSE     Thực phẩm và đồ uống          CT               0   
3       AAS    UPCOM        Dịch vụ tài chính          CK            2087   
4       AAT     HOSE  Hàng cá nhân & Gia dụng          CT            2431   
...     ...      ...                      ...         ...             ...   
1597    XPH    UPCOM  Hàng cá nhân & Gia dụng          CT               0   
1598    YBC    UPCOM     Xây dựng và Vật liệu          CT             254   
1599    YBM     HOSE        Tài nguyên Cơ bản          CT               0   
1600    YEG     HOSE             Truyền thông          CT             310   
1601    YTC    UPCOM                     Y tế          CT             259   

      foreignPercent  outstandingShare  issueShare establishedYear  \
0    

In [43]:
dfInfoDetail= df

In [44]:
dfInfoDetail.columns

Index(['ticker', 'exchange', 'industry', 'companyType', 'noShareholders',
       'foreignPercent', 'outstandingShare', 'issueShare', 'establishedYear',
       'noEmployees', 'stockRating', 'deltaInWeek', 'deltaInMonth',
       'deltaInYear', 'shortName', 'industryEn', 'industryID', 'industryIDv2',
       'website'],
      dtype='object')

In [51]:
dfInfoDetail = dfInfoDetail[['ticker','exchange','shortName','industry','industryEn','establishedYear','noEmployees','noShareholders','foreignPercent','stockRating','outstandingShare','issueShare','website']]
dfInfoDetail['updateAt'] = datetime.now()
dfInfoDetail

,ticker,exchange,shortName,industry,industryEn,establishedYear,noEmployees,noShareholders,foreignPercent,stockRating,outstandingShare,issueShare,website,updateAt
0,A32,UPCOM,Công ty 32,Hàng cá nhân & Gia dụng,Personal & Household Goods,2008,1098,488,0.000,3.6,6.8,6.8,http://32jsc.com,2023-11-01 00:49:37.080771
1,AAA,HOSE,An Phát Bioplastics,Hóa chất,Chemicals,2007,1689,27697,0.021,3.5,382.3,382.3,https://anphatbioplastics.com,2023-11-01 00:49:37.080771
2,AAM,HOSE,Thủy sản Mekong,Thực phẩm và đồ uống,Food & Beverage,2002,159,0,0.008,3.9,10.5,12.3,http://www.mekongfish.vn,2023-11-01 00:49:37.080771
3,AAS,UPCOM,Chứng khoán SmartInvest,Dịch vụ tài chính,Financial Services,2006,43,2087,0.012,4.0,200.0,200.0,http://www.sisi.com.vn,2023-11-01 00:49:37.080771
4,AAT,HOSE,Tập đoàn Tiên Sơn Thanh Hóa,Hàng cá nhân & Gia dụng,Personal & Household Goods,1995,55,2431,0.009,4.3,70.8,70.8,http://tiensonaus.com,2023-11-01 00:49:37.080771
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1597,XPH,UPCOM,Xà phòng Hà Nội,Hàng cá nhân & Gia dụng,Personal & Household Goods,1960,85,0,0.001,3.8,13.0,13.0,https://haso.vn,2023-11-01 00:49:37.080771
1598,YBC,UPCOM,Xi măng và Khoáng sản Yên Bái,Xây dựng và Vật liệu,Construction & Materials,2003,331,254,0.000,3.1,11.8,11.8,http://ybcmjsc.com.vn,2023-11-01 00:49:37.080771
1599,YBM,HOSE,Khoáng sản CN Yên Bái,Tài nguyên Cơ bản,Basic Resources,2015,175,0,0.003,3.7,14.3,14.3,http://www.ybm.com.vn,2023-11-01 00:49:37.080771
1600,YEG,HOSE,Tập đoàn Yeah1,Truyền thông,Media,2006,76,310,0.050,3.4,76.3,76.3,https://www.yeah1group.com,2023-11-01 00:49:37.080771


In [52]:
import pandas as pd
import pyodbc
from sqlalchemy import create_engine


df=dfInfoDetail
df = df.replace('-', 0)


conn = pyodbc.connect('Driver={SQL Server};'
                      'Server=MSI\\SQLEXPRESS;'
                      'Database=dataFinance;'
                      'Trusted_Connection=yes;')

cursor = conn.cursor()

for index, row in df.iterrows():
    cursor.execute("INSERT INTO overviewInfo (ticker, exchange, shortName, industry, industryEn, establishedYear, noEmployees, noShareholders, foreignPercent, stockRating, outstandingShare, issueShare, website,updateAt) VALUES (?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?,?)",
                   row['ticker'], row['exchange'], row['shortName'], row['industry'], row['industryEn'], row['establishedYear'], row['noEmployees'], row['noShareholders'], row['foreignPercent'], row['stockRating'], row['outstandingShare'], row['issueShare'], row['website'], row['updateAt'])

conn.commit()

conn.close()

